In [1]:
import argparse
import json
import numpy as np
import time
import os
import shutil
import psutil
import sys
import socket
import traceback

from collections import OrderedDict

import torch
from torch.utils.data import DataLoader

import robomimic
import robomimic.utils.train_utils as TrainUtils
import robomimic.utils.torch_utils as TorchUtils
import robomimic.utils.obs_utils as ObsUtils
import robomimic.utils.env_utils as EnvUtils
import robomimic.utils.file_utils as FileUtils
from robomimic.config import config_factory
from robomimic.algo import algo_factory, RolloutPolicy
from robomimic.utils.log_utils import PrintLogger, DataLogger, flush_warnings

ROBOMIMIC WARNING(
    No private macro file found!
    It is recommended to use a private macro file
    To setup, run: python /home/franka_deoxys/robomimic/robomimic/scripts/setup_macros.py
)


In [2]:
config_file = "/home/franka_deoxys/data_franka/configs/bc_trans.json" 

In [3]:
ext_cfg = json.load(open(config_file, 'r'))
config = config_factory(ext_cfg["algo_name"])
# update config with external json - this will throw errors if
# the external config has keys not present in the base algo config
with config.values_unlocked():
    config.update(ext_cfg)

config.lock()

# get torch device
device = TorchUtils.get_torch_device(try_to_use_cuda=config.train.cuda)
device

device(type='cuda', index=0)

In [4]:
# read config to set up metadata for observation modalities (e.g. detecting rgb observations)
ObsUtils.initialize_obs_utils_with_config(config)

# make sure the dataset exists
dataset_path = os.path.expanduser(config.train.data)

# load basic metadata from training file
shape_meta = FileUtils.get_shape_metadata_from_dataset(
    dataset_path=config.train.data,
    all_obs_keys=config.all_obs_keys,
    verbose=True
)
trainset, validset = TrainUtils.load_data_for_training(
    config, obs_keys=shape_meta["all_obs_keys"])
train_sampler = trainset.get_dataset_sampler()


============= Initialized Observation Utils with Obs Spec =============

using obs modality: low_dim with keys: ['ee_states', 'gripper_states', 'joint_states']
using obs modality: rgb with keys: ['agentview_rgb', 'eye_in_hand_rgb']
using obs modality: depth with keys: []
using obs modality: scan with keys: []
obs key agentview_rgb with shape (120, 120, 3)
obs key ee_states with shape (16,)
obs key eye_in_hand_rgb with shape (120, 120, 3)
obs key gripper_states with shape (1,)
obs key joint_states with shape (7,)
SequenceDataset: loading dataset into memory...
100%|██████████| 30/30 [00:00<00:00, 195.08it/s]
SequenceDataset: caching get_item calls...
100%|██████████| 3767/3767 [00:01<00:00, 2436.81it/s]


In [5]:
# initialize data loaders
train_loader = DataLoader(
    dataset=trainset,
    sampler=train_sampler,
    batch_size=config.train.batch_size,
    shuffle=(train_sampler is None),
    num_workers=config.train.num_data_workers,
    drop_last=True
)

In [6]:
model = algo_factory(
    algo_name=config.algo_name,
    config=config,
    obs_key_shapes=shape_meta["all_shapes"],
    ac_dim=shape_meta["ac_dim"],
    device=device,
)

/home/franka_deoxys/miniconda3/envs/fr3/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/franka_deoxys/miniconda3/envs/fr3/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Created GPT_Backbone model with number of parameters: 18906112


In [ ]:
# step_log = TrainUtils.run_epoch(
#     model=model,
#     data_loader=train_loader,
#     epoch=epoch,
#     num_steps=train_num_steps,
#     obs_normalization_stats=obs_normalization_stats,
# )
# model.on_epoch_end(epoch)

In [9]:
len(train_loader)

37

In [10]:
batch = next(iter(train_loader))
batch.keys()

dict_keys(['actions', 'obs'])

In [12]:
for key in batch['obs'].keys():
    print(key, batch['obs'][key].shape)

agentview_rgb torch.Size([100, 10, 120, 120, 3])
ee_states torch.Size([100, 10, 16])
eye_in_hand_rgb torch.Size([100, 10, 120, 120, 3])
gripper_states torch.Size([100, 10, 1])
joint_states torch.Size([100, 10, 7])
